# 🚀 Product Matcher v4 — GPU Optimized

**Система сопоставления товаров с использованием нейросетей**

- Encoder: `BGE-M3` (создание эмбеддингов)
- Reranker: `BGE-reranker-v2-m3` (переранжирование кандидатов)
- Index: `FAISS` (быстрый поиск ближайших соседей)

Оптимизирован для GPU с 4GB VRAM (RTX 3050 и подобных).

---
## 1️⃣ Imports & Setup

In [2]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, CrossEncoder
import faiss
import re
from tqdm import tqdm
from typing import List, Dict, Tuple, Optional
import warnings
import gc
import os
from pathlib import Path
import torch

warnings.filterwarnings('ignore')

---
## 2️⃣ GPU Check

In [3]:
def check_gpu_status():
    """Проверка и вывод статуса GPU."""
    print("=" * 60)
    print("🔍 GPU CHECK")
    print("=" * 60)
    
    print(f"\n📦 PyTorch version: {torch.__version__}")
    print(f"🔧 CUDA available: {torch.cuda.is_available()}")
    
    if torch.cuda.is_available():
        print(f"🎮 CUDA version: {torch.version.cuda}")
        print(f"📊 GPU count: {torch.cuda.device_count()}")
        
        for i in range(torch.cuda.device_count()):
            props = torch.cuda.get_device_properties(i)
            print(f"\n   GPU {i}: {props.name}")
            print(f"   Memory: {props.total_memory / 1e9:.1f} GB")
            print(f"   Compute capability: {props.major}.{props.minor}")
        
        print(f"\n💾 Current VRAM usage:")
        print(f"   Allocated: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
        print(f"   Cached: {torch.cuda.memory_reserved() / 1e9:.2f} GB")
    else:
        print("⚠️  GPU unavailable! Using CPU.")
    
    print(f"\n📦 FAISS GPU support: {hasattr(faiss, 'StandardGpuResources')}")
    print("=" * 60)
    
    return torch.cuda.is_available()


# Запуск проверки
GPU_AVAILABLE = check_gpu_status()

🔍 GPU CHECK

📦 PyTorch version: 2.6.0+cu124
🔧 CUDA available: True
🎮 CUDA version: 12.4
📊 GPU count: 1

   GPU 0: NVIDIA GeForce RTX 3050 Laptop GPU
   Memory: 4.3 GB
   Compute capability: 8.6

💾 Current VRAM usage:
   Allocated: 0.00 GB
   Cached: 0.00 GB

📦 FAISS GPU support: False


---
## 3️⃣ ProductMatcherGPU — Core Class

In [4]:
class ProductMatcherGPU:
    """
    GPU-оптимизированный matcher товаров:
    
    Pipeline:
        1. Retrieval: BGE-M3 (GPU) + FAISS → Top-K кандидатов
        2. Reranking: BGE-reranker-v2-m3 (GPU) → Лучший кандидат
    """
    
    # Синглтоны моделей (экономия памяти при повторном использовании)
    _encoder_instance = None
    _reranker_instance = None
    
    # =========================================================================
    # INITIALIZATION
    # =========================================================================
    
    def __init__(self, 
                 encoder_model: str = 'BAAI/bge-m3',
                 reranker_model: str = 'BAAI/bge-reranker-v2-m3',
                 cache_dir: str = './cache',
                 use_fp16: bool = True,
                 gpu_id: int = 0,
                 force_gpu: bool = False):
        """
        Args:
            encoder_model: Модель для эмбеддингов
            reranker_model: Модель для реранкинга
            cache_dir: Папка для кэша
            use_fp16: Использовать float16 (экономит VRAM)
            gpu_id: ID GPU
            force_gpu: Ошибка если GPU недоступен
        """
        self.gpu_available = torch.cuda.is_available()
        self.gpu_id = gpu_id
        
        if force_gpu and not self.gpu_available:
            raise RuntimeError("❌ GPU not found!")
        
        # Атрибуты
        self.cache_dir = Path(cache_dir)
        self.cache_dir.mkdir(exist_ok=True)
        self.faiss_index: Optional[faiss.Index] = None
        self.gpu_resources = None
        self.eva_texts_clean: Optional[List[str]] = None
        self.eva_df: Optional[pd.DataFrame] = None
        
        # Device
        self.device = f'cuda:{gpu_id}' if self.gpu_available else 'cpu'
        if self.gpu_available:
            torch.cuda.set_device(gpu_id)
        
        self._print_init_info(gpu_id, use_fp16)
        
        # Dtype
        self.model_dtype = torch.float16 if (use_fp16 and self.gpu_available) else torch.float32
        
        # Загрузка моделей
        self._load_encoder(encoder_model)
        self._load_reranker(reranker_model)
        
        if self.gpu_available:
            self._print_gpu_memory()
        
        print("\n✅ Initialization complete!")
        print("=" * 60)
    
    def _print_init_info(self, gpu_id: int, use_fp16: bool):
        """Вывод информации об инициализации."""
        print("=" * 60)
        print("🚀 Initializing ProductMatcherGPU")
        print("=" * 60)
        print(f"🖥️  Device: {self.device}")
        
        if self.gpu_available:
            props = torch.cuda.get_device_properties(gpu_id)
            self.vram_total = props.total_memory / 1e9
            print(f"🎮 GPU: {props.name}")
            print(f"💾 VRAM: {self.vram_total:.1f} GB")
            
            # FAISS-GPU
            self.faiss_gpu_available = hasattr(faiss, 'StandardGpuResources')
            if self.faiss_gpu_available:
                try:
                    self.gpu_resources = faiss.StandardGpuResources()
                    self.gpu_resources.setTempMemory(256 * 1024 * 1024)
                    print(f"🔧 FAISS-GPU: ✅ Available")
                except:
                    self.faiss_gpu_available = False
                    print(f"🔧 FAISS-GPU: ❌ Initialization error")
            else:
                print(f"🔧 FAISS-GPU: ❌ Not installed (Using CPU)")
            
            print(f"⚡ Precision: {'FP16' if use_fp16 else 'FP32'}")
        else:
            self.vram_total = 0
            self.faiss_gpu_available = False
    
    def _load_encoder(self, model_name: str):
        """Загрузка encoder на GPU."""
        if ProductMatcherGPU._encoder_instance is None:
            print(f"\n📥 Loading encoder: {model_name}")
            
            try:
                ProductMatcherGPU._encoder_instance = SentenceTransformer(
                    model_name,
                    device=self.device,
                    model_kwargs={'torch_dtype': self.model_dtype}
                )
            except Exception:
                ProductMatcherGPU._encoder_instance = SentenceTransformer(model_name, device=self.device)
                if self.model_dtype == torch.float16 and self.gpu_available:
                    ProductMatcherGPU._encoder_instance.half()
            
            if self.gpu_available:
                try:
                    dev = next(ProductMatcherGPU._encoder_instance.parameters()).device
                    print(f"   ✅ Encoder on: {dev}")
                except:
                    pass
        else:
            print("✅ Encoder already loaded")
        
        self.encoder = ProductMatcherGPU._encoder_instance
    
    def _load_reranker(self, model_name: str):
        """Загрузка reranker на GPU."""
        if ProductMatcherGPU._reranker_instance is None:
            print(f"\n📥 Loading reranker: {model_name}")
            
            try:
                ProductMatcherGPU._reranker_instance = CrossEncoder(
                    model_name,
                    max_length=512,
                    device=self.device,
                    model_kwargs={'torch_dtype': self.model_dtype}
                )
            except Exception:
                ProductMatcherGPU._reranker_instance = CrossEncoder(
                    model_name, max_length=512, device=self.device
                )
                if self.model_dtype == torch.float16 and self.gpu_available:
                    ProductMatcherGPU._reranker_instance.model.half()
            
            if self.gpu_available:
                try:
                    dev = next(ProductMatcherGPU._reranker_instance.model.parameters()).device
                    print(f"   ✅ Reranker on: {dev}")
                except:
                    pass
        else:
            print("✅ Reranker already loaded")
        
        self.reranker = ProductMatcherGPU._reranker_instance
    
    # =========================================================================
    # GPU UTILITIES
    # =========================================================================
    
    def _print_gpu_memory(self):
        """Вывод использования GPU памяти."""
        if self.gpu_available:
            allocated = torch.cuda.memory_allocated(self.gpu_id) / 1e9
            reserved = torch.cuda.memory_reserved(self.gpu_id) / 1e9
            print(f"\n💾 GPU Memory: {allocated:.2f} / {self.vram_total:.1f} GB (reserved: {reserved:.2f} GB)")
    
    def _get_free_vram(self) -> float:
        """Получить свободную VRAM в GB."""
        if not self.gpu_available:
            return 0
        allocated = torch.cuda.memory_allocated(self.gpu_id) / 1e9
        return self.vram_total - allocated
    
    @classmethod
    def clear_models(cls):
        """Очистка моделей и GPU памяти."""
        print("🧹 Clearing models...")
        cls._encoder_instance = None
        cls._reranker_instance = None
        gc.collect()
        
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            torch.cuda.synchronize()
            print(f"   GPU memory: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
        
        print("✅ Cleared")
    
    # =========================================================================
    # TEXT CLEANING
    # =========================================================================
    
    def clean_text(self, text: str) -> str:
        """Нормализация текста товара."""
        if pd.isna(text):
            return ""
        
        text = str(text).lower().strip()
        
        # Нормализация единиц измерения
        replacements = [
            (r'(\d+)\s*мл\b', r'\1мл'),
            (r'(\d+)\s*ml\b', r'\1мл'),
            (r'(\d+)\s*г\b', r'\1г'),
            (r'(\d+)\s*g\b', r'\1г'),
            (r'(\d+)\s*кг\b', r'\1кг'),
            (r'(\d+)\s*kg\b', r'\1кг'),
            (r'(\d+)\s*л\b', r'\1л'),
            (r'(\d+)\s*l\b', r'\1л'),
            (r'(\d+)\s*шт\b', r'\1шт'),
        ]
        
        for pattern, repl in replacements:
            text = re.sub(pattern, repl, text)
        
        # Очистка спецсимволов
        text = re.sub(r'[^\w\s\-\.,/]', ' ', text)
        text = re.sub(r'\s+', ' ', text)
        
        return text.strip()
    
    def clean_texts_batch(self, texts: List[str], desc: str = "Cleaning") -> List[str]:
        """Батчевая очистка текстов."""
        return [self.clean_text(t) for t in tqdm(texts, desc=desc)]
    
    # =========================================================================
    # EMBEDDINGS
    # =========================================================================
    
    def create_embeddings(self, 
                          texts: List[str], 
                          batch_size: int = 8,
                          show_progress: bool = True) -> np.ndarray:
        """Создание эмбеддингов (GPU)."""
        
        # Авто-подбор batch_size под VRAM
        if self.gpu_available:
            free_vram = self._get_free_vram()
            if free_vram < 1.5:
                batch_size = min(batch_size, 4)
            elif free_vram < 2.0:
                batch_size = min(batch_size, 8)
            elif free_vram < 3.0:
                batch_size = min(batch_size, 16)
            
            print(f"   Batch size: {batch_size} (free VRAM: {free_vram:.1f} GB)")
        
        embeddings = self.encoder.encode(
            texts,
            batch_size=batch_size,
            show_progress_bar=show_progress,
            normalize_embeddings=True,
            convert_to_numpy=True,
            device=self.device
        )
        
        return embeddings.astype('float32')
    
    # =========================================================================
    # FAISS INDEX
    # =========================================================================
    
    def create_faiss_index(self, embeddings: np.ndarray) -> faiss.Index:
        """Создание FAISS индекса."""
        dimension = embeddings.shape[1]
        
        # CPU индекс (более надёжный)
        index = faiss.IndexFlatIP(dimension)
        index.add(embeddings)
        
        # GPU индекс если доступен
        if self.faiss_gpu_available and self.gpu_resources and self._get_free_vram() > 0.5:
            try:
                gpu_index = faiss.index_cpu_to_gpu(self.gpu_resources, self.gpu_id, index)
                print(f"   ✅ FAISS on GPU")
                return gpu_index
            except Exception as e:
                print(f"   ⚠️ FAISS on GPU failed: {e}")
        
        print(f"   📊 FAISS on CPU")
        return index
    
    def _index_to_cpu(self, index: faiss.Index) -> faiss.Index:
        """Перенос индекса на CPU."""
        if hasattr(faiss, 'index_gpu_to_cpu'):
            try:
                return faiss.index_gpu_to_cpu(index)
            except:
                pass
        return index
    
    # =========================================================================
    # CACHE MANAGEMENT
    # =========================================================================
    
    def _get_cache_path(self, name: str) -> Path:
        return self.cache_dir / name
    
    def save_index(self, 
                   df: pd.DataFrame,
                   text_column: str,
                   cache_name: str = 'products'):
        """Создание и сохранение индекса."""
        print("\n" + "=" * 60)
        print(f"💾 Creating Index ({len(df)} items)")
        print("=" * 60)
        
        print("\n1️⃣ Cleaning texts...")
        texts_clean = self.clean_texts_batch(df[text_column].tolist())
        
        print("\n2️⃣ Creating embeddings...")
        embeddings = self.create_embeddings(texts_clean)
        
        print("\n3️⃣ Creating FAISS index...")
        index = self.create_faiss_index(embeddings)
        
        print("\n4️⃣ Saving...")
        cpu_index = self._index_to_cpu(index)
        
        np.save(self._get_cache_path(f'{cache_name}_embeddings.npy'), embeddings)
        pd.DataFrame({'text_clean': texts_clean}).to_parquet(
            self._get_cache_path(f'{cache_name}_texts.parquet')
        )
        faiss.write_index(cpu_index, str(self._get_cache_path(f'{cache_name}_index.faiss')))
        df.to_parquet(self._get_cache_path(f'{cache_name}_products.parquet'))
        
        print(f"✅ Saved to {self.cache_dir}/")
        
        self.faiss_index = index
        self.eva_texts_clean = texts_clean
        self.eva_df = df
        
        if self.gpu_available:
            self._print_gpu_memory()
        
        return index, texts_clean
    
    def load_index(self, cache_name: str = 'products') -> bool:
        """Загрузка индекса из кэша."""
        paths = [
            self._get_cache_path(f'{cache_name}_embeddings.npy'),
            self._get_cache_path(f'{cache_name}_texts.parquet'),
            self._get_cache_path(f'{cache_name}_index.faiss'),
            self._get_cache_path(f'{cache_name}_products.parquet'),
        ]
        
        if not all(p.exists() for p in paths):
            print("⚠️ Cache not found")
            return False
        
        print("\n📂 Loading from cache...")
        
        self.faiss_index = faiss.read_index(str(paths[2]))
        self.eva_texts_clean = pd.read_parquet(paths[1])['text_clean'].tolist()
        self.eva_df = pd.read_parquet(paths[3])
        
        print(f"✅ Loaded {self.faiss_index.ntotal} items")
        
        if self.gpu_available:
            self._print_gpu_memory()
        
        return True
    
    # =========================================================================
    # SEARCH & RERANKING
    # =========================================================================
    
    def search_candidates(self, 
                          query_embeddings: np.ndarray,
                          top_k: int = 10) -> Tuple[np.ndarray, np.ndarray]:
        """Поиск кандидатов."""
        scores, indices = self.faiss_index.search(query_embeddings.astype('float32'), top_k)
        return indices, scores
    
    def rerank_batch(self,
                     query_texts: List[str],
                     candidates_indices: np.ndarray,
                     batch_size: int = 16) -> List[Tuple[int, float, int]]:
        """Реранкинг кандидатов."""
        
        # Сбор пар
        all_pairs = []
        pair_info = []
        
        for q_idx, (query, cand_indices) in enumerate(zip(query_texts, candidates_indices)):
            for pos, idx in enumerate(cand_indices):
                if idx >= 0:
                    all_pairs.append([query, self.eva_texts_clean[idx]])
                    pair_info.append((q_idx, pos, idx))
        
        # Авто-подбор batch_size
        if self.gpu_available:
            free_vram = self._get_free_vram()
            if free_vram < 1.0:
                batch_size = min(batch_size, 8)
            elif free_vram < 1.5:
                batch_size = min(batch_size, 16)
            elif free_vram < 2.0:
                batch_size = min(batch_size, 32)
        
        print(f"   Reranking {len(all_pairs)} pairs (batch={batch_size})...")
        
        # Предсказание
        all_scores = self.reranker.predict(
            all_pairs, 
            batch_size=batch_size,
            show_progress_bar=True
        )
        
        # Группировка
        query_results = {}
        for (q_idx, pos, idx), score in zip(pair_info, all_scores):
            if q_idx not in query_results:
                query_results[q_idx] = []
            query_results[q_idx].append((idx, float(score), pos))
        
        # Выбор лучшего для каждого
        results = []
        for q_idx in range(len(query_texts)):
            if q_idx in query_results:
                best = max(query_results[q_idx], key=lambda x: x[1])
                results.append(best)
            else:
                results.append((-1, 0.0, -1))
        
        return results
    
    # =========================================================================
    # MAIN MATCHING METHOD
    # =========================================================================
    
    def match_products(self,
                       competitor_df: pd.DataFrame,
                       competitor_col: str = 'name',
                       our_df: Optional[pd.DataFrame] = None,
                       our_col: str = 'name',
                       top_k: int = 10,
                       threshold: float = 0.5,
                       encoder_batch: int = 8,
                       reranker_batch: int = 16,
                       use_cache: bool = True,
                       cache_name: str = 'products') -> pd.DataFrame:
        """
        Сопоставление товаров.
        
        Args:
            competitor_df: DataFrame конкурента
            competitor_col: Колонка с названием товара
            our_df: Наш DataFrame (или None если закэширован)
            our_col: Колонка с названием товара
            top_k: Количество кандидатов для реранкинга
            threshold: Порог уверенности
            encoder_batch: Batch size для encoder
            reranker_batch: Batch size для reranker
            use_cache: Использовать кэш
            cache_name: Имя кэша
        """
        print("\n" + "=" * 70)
        print("🚀 MATCHING PRODUCTS")
        print("=" * 70)
        
        if self.gpu_available:
            self._print_gpu_memory()
        
        # 1. Индексация наших товаров
        if self.faiss_index is None:
            if use_cache and self.load_index(cache_name):
                pass
            elif our_df is not None:
                self.save_index(our_df, our_col, cache_name)
            else:
                raise ValueError("Provide our_df or use cache")
        
        print(f"\n📦 Our products: {self.faiss_index.ntotal}")
        print(f"📦 Competitor products: {len(competitor_df)}")
        
        # 2. Обработка товаров конкурента
        print("\n" + "-" * 50)
        print("📝 Stage 1: Preparing competitor products")
        print("-" * 50)
        
        comp_texts = competitor_df[competitor_col].tolist()
        comp_texts_clean = self.clean_texts_batch(comp_texts, "Cleaning")
        
        print("\nCreating embeddings...")
        comp_embeddings = self.create_embeddings(comp_texts_clean, batch_size=encoder_batch)
        
        # 3. Retrieval
        print("\n" + "-" * 50)
        print(f"🔍 Stage 2: Retrieval (Top-{top_k})")
        print("-" * 50)
        
        cand_indices, cand_scores = self.search_candidates(comp_embeddings, top_k)
        print(f"   ✅ Candidates found for {len(cand_indices)} items")
        
        # 4. Reranking
        print("\n" + "-" * 50)
        print("🎯 Stage 3: Reranking")
        print("-" * 50)
        
        rerank_results = self.rerank_batch(comp_texts_clean, cand_indices, batch_size=reranker_batch)
        
        # 5. Результаты
        print("\n" + "-" * 50)
        print("📊 Stage 4: Building results")
        print("-" * 50)
        
        eva_col = our_col if our_col in self.eva_df.columns else self.eva_df.columns[1]
        
        results = []
        for comp_idx, (eva_idx, score, rank) in enumerate(rerank_results):
            status = "matched" if score >= threshold else "low_confidence"
            
            results.append({
                'competitor_index': comp_idx,
                'competitor_product': competitor_df.iloc[comp_idx][competitor_col],
                'our_index': eva_idx if status == "matched" else None,
                'our_product': self.eva_df.iloc[eva_idx][eva_col] if eva_idx >= 0 else None,
                'retrieval_score': float(cand_scores[comp_idx][rank]) if rank >= 0 else 0.0,
                'rerank_score': score,
                'retrieval_rank': rank + 1 if rank >= 0 else -1,
                'match_status': status
            })
        
        result_df = pd.DataFrame(results)
        
        # Статистика
        matched = len(result_df[result_df['match_status'] == 'matched'])
        low_conf = len(result_df[result_df['match_status'] == 'low_confidence'])
        
        print(f"\n✅ Done!")
        print(f"   Matched: {matched} ({matched/len(result_df)*100:.1f}%)")
        print(f"   Low confidence: {low_conf} ({low_conf/len(result_df)*100:.1f}%)")
        
        if self.gpu_available:
            self._print_gpu_memory()
        
        return result_df
    
    # =========================================================================
    # ANALYSIS
    # =========================================================================
    
    def analyze_results(self, df: pd.DataFrame, show_examples: int = 5) -> Dict:
        """Анализ результатов."""
        print("\n" + "=" * 70)
        print("📊 RESULTS ANALYSIS")
        print("=" * 70)
        
        total = len(df)
        matched = df[df['match_status'] == 'matched']
        low_conf = df[df['match_status'] == 'low_confidence']
        
        print(f"\n📈 Statistics:")
        print(f"   Total: {total}")
        print(f"   ✅ Matched: {len(matched)} ({len(matched)/total*100:.1f}%)")
        print(f"   ⚠️  Low confidence: {len(low_conf)} ({len(low_conf)/total*100:.1f}%)")
        
        print(f"\n📉 Rerank score:")
        print(f"   Mean: {df['rerank_score'].mean():.4f}")
        print(f"   Median: {df['rerank_score'].median():.4f}")
        print(f"   Min/Max: {df['rerank_score'].min():.4f} / {df['rerank_score'].max():.4f}")
        
        print(f"\n📊 Distribution:")
        for t in [0.9, 0.7, 0.5, 0.3]:
            count = len(df[df['rerank_score'] >= t])
            print(f"   >= {t}: {count} ({count/total*100:.1f}%)")
        
        if show_examples > 0 and len(matched) > 0:
            print(f"\n📋 Top-{show_examples} Best Matches:")
            print("-" * 70)
            
            for i, (_, row) in enumerate(matched.nlargest(show_examples, 'rerank_score').iterrows(), 1):
                print(f"\n{i}. Score: {row['rerank_score']:.4f}")
                print(f"   Competitor: {row['competitor_product'][:65]}")
                print(f"   Our:        {str(row['our_product'])[:65]}")
        
        if show_examples > 0 and len(low_conf) > 0:
            print(f"\n⚠️  Worst {min(3, len(low_conf))}:")
            print("-" * 70)
            
            for i, (_, row) in enumerate(low_conf.nsmallest(min(3, len(low_conf)), 'rerank_score').iterrows(), 1):
                print(f"\n{i}. Score: {row['rerank_score']:.4f}")
                print(f"   Competitor: {row['competitor_product'][:65]}")
                print(f"   Best Guess: {str(row['our_product'])[:65]}")
        
        return {
            'total': total,
            'matched': len(matched),
            'low_confidence': len(low_conf),
            'avg_score': df['rerank_score'].mean(),
            'median_score': df['rerank_score'].median()
        }

---
## 4️⃣ Quick Start Helper

In [5]:
def quick_match(competitor_csv: str,
                our_csv: str,
                output_csv: str = 'matches.csv',
                competitor_col: str = 'name',
                our_col: str = 'name',
                threshold: float = 0.5,
                top_k: int = 5) -> pd.DataFrame:
    """
    Быстрое сопоставление из CSV файлов.
    
    Пример:
        results = quick_match('competitor.csv', 'our_products.csv')
    """
    print("📂 Loading data...")
    competitor_df = pd.read_csv(competitor_csv)
    our_df = pd.read_csv(our_csv)
    
    print(f"   Competitor: {len(competitor_df)} items")
    print(f"   Our: {len(our_df)} items")
    
    matcher = ProductMatcherGPU(use_fp16=True, force_gpu=False)
    
    results = matcher.match_products(
        competitor_df,
        competitor_col=competitor_col,
        our_df=our_df,
        our_col=our_col,
        top_k=top_k,
        threshold=threshold,
        encoder_batch=8,
        reranker_batch=16,
        use_cache=True
    )
    
    matcher.analyze_results(results)
    
    results.to_csv(output_csv, index=False, encoding='utf-8-sig')
    print(f"\n💾 Saved: {output_csv}")
    
    return results

---
## 5️⃣ Configuration

In [6]:
# =============================================================================
# ⚙️ НАСТРОЙКИ (РЕДАКТИРУЙ ЗДЕСЬ)
# =============================================================================

# Файлы
COMPETITOR_FILE = "competitor_products.csv"
OUR_FILE = "our_products.csv"
OUTPUT_FILE = "results_gpu_matched.csv"

# Колонки с названиями товаров
COMPETITOR_COL_NAME = 'name'
OUR_COL_NAME = 'name'

# Параметры производительности (для 4GB VRAM)
USE_FP16 = True                # FP16 экономит память
BATCH_SIZE_ENCODER = 8         # 4-8 для 4GB VRAM
BATCH_SIZE_RERANKER = 16       # 16 для 4GB VRAM
TOP_K_CANDIDATES = 5           # Кандидатов для реранкинга
THRESHOLD = 0.5                # Порог уверенности (0.0 - 1.0)

---
## 6️⃣ Load Data

In [7]:
print("📂 LOADING DATA:")

try:
    # Для CSV с точкой с запятой: добавь sep=';'
    df_competitor = pd.read_csv(COMPETITOR_FILE)
    df_our = pd.read_csv(OUR_FILE)
    
    print(f"✅ Competitor: {len(df_competitor)} items")
    print(f"✅ Our: {len(df_our)} items")
    
except FileNotFoundError as e:
    print(f"❌ Error: {e}")
    print("   Убедись, что файлы находятся в директории скрипта.")

📂 LOADING DATA:
✅ Competitor: 500 items
✅ Our: 500 items


In [8]:
# Проверка колонок
print("🔍 COLUMN ANALYSIS:")
print(f"   Competitor: {df_competitor.columns.tolist()}")
print(f"   Our:        {df_our.columns.tolist()}")

🔍 COLUMN ANALYSIS:
   Competitor: ['id', 'name']
   Our:        ['id', 'name']


---
## 7️⃣ Initialize Matcher

In [9]:
matcher = ProductMatcherGPU(
    use_fp16=USE_FP16,
    cache_dir='./cache_products'
)

🚀 Initializing ProductMatcherGPU
🖥️  Device: cuda:0
🎮 GPU: NVIDIA GeForce RTX 3050 Laptop GPU
💾 VRAM: 4.3 GB
🔧 FAISS-GPU: ❌ Not installed (Using CPU)
⚡ Precision: FP16

📥 Loading encoder: BAAI/bge-m3


`torch_dtype` is deprecated! Use `dtype` instead!


   ✅ Encoder on: cuda:0

📥 Loading reranker: BAAI/bge-reranker-v2-m3
   ✅ Reranker on: cuda:0

💾 GPU Memory: 2.27 / 4.3 GB (reserved: 2.28 GB)

✅ Initialization complete!


---
## 8️⃣ Run Matching

In [10]:
try:
    results = matcher.match_products(
        competitor_df=df_competitor,
        competitor_col=COMPETITOR_COL_NAME,
        our_df=df_our,
        our_col=OUR_COL_NAME,
        top_k=TOP_K_CANDIDATES,
        threshold=THRESHOLD,
        encoder_batch=BATCH_SIZE_ENCODER,
        reranker_batch=BATCH_SIZE_RERANKER,
        use_cache=True
    )
    
except KeyError as e:
    print(f"\n❌ COLUMN ERROR: Column not found {e}")
    print("   Проверь настройки COMPETITOR_COL_NAME и OUR_COL_NAME")


🚀 MATCHING PRODUCTS

💾 GPU Memory: 2.27 / 4.3 GB (reserved: 2.28 GB)

📂 Loading from cache...
✅ Loaded 500 items

💾 GPU Memory: 2.27 / 4.3 GB (reserved: 2.28 GB)

📦 Our products: 500
📦 Competitor products: 500

--------------------------------------------------
📝 Stage 1: Preparing competitor products
--------------------------------------------------


Cleaning: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 27845.82it/s]


Creating embeddings...
   Batch size: 8 (free VRAM: 2.0 GB)


Batches:   0%|          | 0/63 [00:00<?, ?it/s]


--------------------------------------------------
🔍 Stage 2: Retrieval (Top-5)
--------------------------------------------------
   ✅ Candidates found for 500 items

--------------------------------------------------
🎯 Stage 3: Reranking
--------------------------------------------------
   Reranking 2500 pairs (batch=16)...


Batches:   0%|          | 0/157 [00:00<?, ?it/s]


--------------------------------------------------
📊 Stage 4: Building results
--------------------------------------------------

✅ Done!
   Matched: 500 (100.0%)
   Low confidence: 0 (0.0%)

💾 GPU Memory: 2.28 / 4.3 GB (reserved: 2.33 GB)


---
## 9️⃣ Analyze & Save Results

In [11]:
# Анализ
stats = matcher.analyze_results(results, show_examples=5)


📊 RESULTS ANALYSIS

📈 Statistics:
   Total: 500
   ✅ Matched: 500 (100.0%)
   ⚠️  Low confidence: 0 (0.0%)

📉 Rerank score:
   Mean: 0.9911
   Median: 0.9995
   Min/Max: 0.7734 / 1.0000

📊 Distribution:
   >= 0.9: 489 (97.8%)
   >= 0.7: 500 (100.0%)
   >= 0.5: 500 (100.0%)
   >= 0.3: 500 (100.0%)

📋 Top-5 Best Matches:
----------------------------------------------------------------------

1. Score: 1.0000
   Competitor: Pantene ProV Шампунь объем 250мл
   Our:        Pantene Pro-V Shamp объем 250мл

2. Score: 1.0000
   Competitor: L'Oreal Elseve ШАМПУНЬ против выпадения 400мл
   Our:        Shamp Эльсев - против выпадения / 400мл

3. Score: 1.0000
   Competitor: ШАМПУНЬ Elseve для окрашенных волос 250мл
   Our:        для окрашенных волос Засіб для миття волосся Elseve 250мл

4. Score: 1.0000
   Competitor: ШАМПУНЬ SCHAUMA "для мужчин" 400мл
   Our:        для мужчин Шампунь для волосся SCHAUMA 400мл

5. Score: 1.0000
   Competitor: Шампунь д/волос Шаума (fresh it up) 250мл
   Our:  

In [12]:
# Сохранение
results.to_csv(OUTPUT_FILE, index=False, encoding='utf-8-sig')
print(f"💾 Results saved to: {OUTPUT_FILE}")

💾 Results saved to: results_gpu_matched.csv


In [13]:
# Превью результатов
results.head(10)

,competitor_index,competitor_product,our_index,our_product,retrieval_score,rerank_score,retrieval_rank,match_status
0,0,Хэд энд Шолдерс - Шампунь д/волос против перхо...,183,SHAMPOO [H&S] против перхоти (400мл),0.795066,0.999512,1,matched
1,1,Head & Shoulders Засіб д/волосся ментол 200мл,387,Hair cleanser Head&Shoulders - ментол / 200мл,0.882091,0.998047,1,matched
2,2,Head&Shoulders - Шамп. для чувствительной кожи...,35,Hair cleanser [Head&Shoulders] для чувствитель...,0.889190,0.999512,1,matched
3,3,"Pantene ProV - Засіб д/волосся восстановление,...",438,Hair cleanser [Пантин] восстановление (400мл),0.779750,0.989258,1,matched
4,4,Pantene ProV Шампунь объем 250мл,39,Pantene Pro-V Shamp объем 250мл,0.876570,1.000000,1,matched
5,5,Шамп. Pantene (питание) 200мл,10,питание Shamp Pantene 200мл,0.922740,0.999512,1,matched
6,6,L'Oreal Elseve ШАМПУНЬ против выпадения 400мл,18,Shamp Эльсев - против выпадения / 400мл,0.819458,1.000000,1,matched
7,7,ШАМПУНЬ Elseve для окрашенных волос 250мл,82,для окрашенных волос Засіб для миття волосся E...,0.927136,1.000000,1,matched
8,8,Засіб д/волосся L'Oreal Elseve (экстраординарн...,219,Шампунь для волосся [Elseve L'Oreal] экстраорд...,0.938586,0.997559,1,matched
9,9,Gliss Шамп. экстремальное восстановление 400мл,332,Глисс Кур | Шампунь для волосся | экстремально...,0.843782,0.996582,2,matched


---
## 🧹 Cleanup

In [13]:
# Очистка GPU памяти (запускай после завершения работы)
ProductMatcherGPU.clear_models()

🧹 Clearing models...
   GPU memory: 2.28 GB
✅ Cleared
